In [19]:
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib_fontja

# データベースを接続
conn = sqlite3.connect('suumo.db')
c = conn.cursor()

# データを取得
c.execute('SELECT * FROM suumo')
data = c.fetchall()

# データをDataFrameに変換
df = pd.DataFrame(data, columns=['id', 'property', 'property_name', 'location', 'price', 'Manage_fee', 'deposit', 'gratuity', 'Security_deposit', 'expense', 'floor_plan', 'buildings_age', 'floor', 'building_age', 'url'])

# データベースを切断
conn.close()

# データの表示
df.head(5)

,id,property,property_name,location,price,Manage_fee,deposit,gratuity,Security_deposit,expense,floor_plan,buildings_age,floor,building_age,url
0,1,ＪＲ山手線 高田馬場駅 3階建 築7年,ＪＲ山手線 高田馬場駅 3階建 築7年,東京都新宿区高田馬場４,228000.0,6000,228000.0,456000.0,None,NaN,2LDK,7,3階,マンション,https://suumo.jp/chintai/jnc_000095516930/?bc=...
1,2,都営大江戸線 牛込神楽坂駅 4階建 築2年,都営大江戸線 牛込神楽坂駅 4階建 築2年,東京都新宿区南山伏町,264000.0,6000,264000.0,264000.0,None,NaN,2LDK,2,1階,マンション,https://suumo.jp/chintai/jnc_000095906984/?bc=...
2,3,ザ・パークハウス西新宿タワー60,ザ・パークハウス西新宿タワー60 4102号室,東京都新宿区西新宿５,210000.0,25000,NaN,210000.0,None,NaN,1DK,8,41階,マンション,https://suumo.jp/chintai/jnc_000094825097/?bc=...
3,4,ザ・パークハウス西新宿タワー60,ザ・パークハウス西新宿タワー６０,東京都新宿区西新宿５,220000.0,15000,220000.0,110000.0,None,NaN,1LDK,8,11階,マンション,https://suumo.jp/chintai/jnc_000095894351/?bc=...
4,5,ザ・パークハウス西新宿タワー60,ザ・パークハウス西新宿タワー60 1216号室,東京都新宿区西新宿５,256000.0,None,256000.0,256000.0,None,NaN,1LDK,8,12階,マンション,https://suumo.jp/chintai/jnc_000095177147/?bc=...


In [20]:
# priceとlocationのみを抽出
df = df[['location', 'price']]
df.head(5)

,location,price
0,東京都新宿区高田馬場４,228000.0
1,東京都新宿区南山伏町,264000.0
2,東京都新宿区西新宿５,210000.0
3,東京都新宿区西新宿５,220000.0
4,東京都新宿区西新宿５,256000.0


In [23]:
# propertyが同じ物件をグループ化
df_grouped = df.groupby('location').aggregate(lambda x: ','.join(map(str, x)))
print(df_grouped)

                                                         price
location                                                      
東京都新宿区三栄町    300000.0,144000.0,144000.0,168000.0,169000.0,1...
東京都新宿区上落合１   75000.0,54000.0,180000.0,112000.0,112000.0,610...
東京都新宿区上落合２   113000.0,93000.0,100000.0,360000.0,360000.0,36...
東京都新宿区上落合３   84000.0,93000.0,95000.0,98000.0,105000.0,77000...
東京都新宿区下宮比町         138000.0,115000.0,138000.0,80000.0,110000.0
...                                                        ...
東京都新宿区馬場下町   118000.0,118000.0,92000.0,87000.0,87000.0,9200...
東京都新宿区高田馬場１  154000.0,155000.0,157000.0,158000.0,159000.0,1...
東京都新宿区高田馬場２  111000.0,115000.0,116000.0,131000.0,235000.0,2...
東京都新宿区高田馬場３  90000.0,71000.0,72000.0,68000.0,68000.0,64500....
東京都新宿区高田馬場４  228000.0,228000.0,110000.0,99000.0,122000.0,13...

[142 rows x 1 columns]


In [10]:
# 平均値を計算
df_mean = df_grouped.mean()
df_mean

,price
location,
東京都新宿区三栄町,182923.076923
東京都新宿区上落合１,92956.896552
東京都新宿区上落合２,113289.855072
東京都新宿区上落合３,100333.333333
東京都新宿区下宮比町,116200.000000
...,...
東京都新宿区馬場下町,97533.333333
東京都新宿区高田馬場１,122212.903226
東京都新宿区高田馬場２,149838.709677


In [21]:
# 最大値を計算
df_max = df_grouped.max()

# データの表示
df_max.head(5)

,price
location,
東京都新宿区三栄町,300000.0
東京都新宿区上落合１,290000.0
東京都新宿区上落合２,360000.0
東京都新宿区上落合３,358000.0
東京都新宿区下宮比町,138000.0


In [22]:
# 最小値を計算
df_min = df_grouped.min()

# データの表示
df_min.head(5)

,price
location,
東京都新宿区三栄町,76000.0
東京都新宿区上落合１,52000.0
東京都新宿区上落合２,40000.0
東京都新宿区上落合３,33000.0
東京都新宿区下宮比町,80000.0


In [14]:
#算出した平均値、最大値、最小値を使って箱ひげ図を作成
plt.figure(figsize=(20, 6))
sns.boxplot(x='location', y='price', data=df)
plt.xticks(rotation=90)
plt.show()


TypeError: Data source must be a DataFrame or Mapping, not <class 'pandas.core.groupby.generic.DataFrameGroupBy'>.

<Figure size 2000x600 with 0 Axes>